In [ ]:
import importlib
import performance_shr
importlib.reload(performance_shr)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from performance_shr import convert_to_float, convert_to_bool, convert_to_arcsec, plot_formating

In [ ]:
drift_filename = '../driver/alpaca.csv'

drift_converters_dict = {
    'Dataset': str,   
    'Time': convert_to_float,   
    'TrackingT0': convert_to_bool,
    'TrackingT1': convert_to_bool,
    'TargetRA': convert_to_float,  
    'TargetDEC': convert_to_float,  
    'DriftErrRA': convert_to_arcsec,  
    'DriftErrDec': convert_to_arcsec,  

}

drift = pd.read_csv(drift_filename,converters=drift_converters_dict)
drift.columns = ['Log'] + drift.columns[1:].tolist()
drift = drift[drift.TrackingT0 & drift.TrackingT1].reset_index(drop=True)
drift["RAdeg"] = (drift.TargetRA-6)/24*360 % 360
drift["DriftMagnitude"] = np.sqrt(drift['DriftErrRA']**2 + drift['DriftErrDec']**2)
drift['TimeMin'] = (drift.Time / 60).round(1)
d_ra=np.mean(drift["DriftErrRA"])
d_dec=np.mean(drift["DriftErrDec"])
d_mag=np.mean(drift["DriftMagnitude"])
m_ra=np.std(drift["DriftErrRA"])
m_dec=np.std(drift["DriftErrDec"])
m_mag=np.std(drift["DriftMagnitude"])
count=len(drift[drift.Dataset=='DATA2'])
print(f"Count of Drift Error entries = {count}")
print(f"Average Drift Error (arc-seconds per minute) RA={d_ra:7.2f}, Dec={d_dec:7.2f}, Magnitude={d_mag:7.2f}")
print(f"Stdev   Drift Error (arc-seconds per minute) RA={m_ra:7.2f}, Dec={m_dec:7.2f}, Magnitude={m_mag:7.2f}")
drift[drift.Dataset=='DATA2'][2:5]
drift

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=drift.Time, y=drift.DriftErrRA, name='RA Drift Error', mode='markers'))
fig.add_trace(go.Scatter(x=drift.Time, y=drift.DriftErrDec, name='Dec Drift Error', mode='markers'))

title,labels,xtitle,ytitle,bgcolor = plot_formating('Drift Error vs Elapsed Time','Elapsed Time (seconds)','Drift Error (arc seconds per min)')
fig.update_layout(title=title, xaxis_title=xtitle, yaxis_title=ytitle, plot_bgcolor=bgcolor)

fig.show()

In [ ]:
if (len(drift)):
    fig=ff.create_quiver(x=drift.RAdeg, y=drift.TargetDec, u=drift.DriftErrRA, v=drift.DriftErrDec, scale=3, arrow_scale=0.2)
    title,labels,xtitle,ytitle,bgcolor = plot_formating('Drift Error (arc-seconds per min) vs RA/Dec Position','Right Ascension Axis','Declination Axis ')
    fig.update_layout(title=title, xaxis_title=xtitle, yaxis_title=ytitle, plot_bgcolor=bgcolor, xaxis=dict(dtick=30), yaxis=dict(dtick=15))
    # Add numeric labels
    annotations = []
    for i in range(len(drift)):
        annotations.append(
            go.layout.Annotation(
                x=drift['RAdeg'][i]+4 ,
                y=drift['TargetDec'][i],
                text=f'{drift['DriftMagnitude'][i]:3.1f}',
                showarrow=False,
                font=dict(size=12, color='black')
            )
        )

    fig.update_layout(annotations=annotations)
    fig.show()